 Analyse CO2 vs Temperature Anomaly from 800kyrs – present.
• Examine one other (paleo/modern) features (e.g. CH4 or polar ice-coverage)
• Examine Irish context:
o Climate change signals: (see Maynooth study: The emergence of a climate change 
signal in long-term Irish meteorological observations - ScienceDirect)
• Fuse and analyse data from various data sources and format fused data set as a pandas 
dataframe and export to csv and json formats
• For all of the above variables, analyse the data, the trends and the relationships between 
them (temporal leads/lags/frequency analysis).
• Predict global temperature anomaly over next few decades (synthesise data) and compare to 
published climate models if atmospheric CO2 trends continue
• Comment on accelerated warming based on very latest features (e.g. temperature/polar-icecoverage)


In [449]:
import numpy as np  
import pandas as pd
import csv



## CO2 dataset

This excel document contains data from 800k years in the past up to 2001. 
(Further adjustment to the data file will be completed later to include data up to 2022).

This data includes updated data for CO2 composite with the removal of analytical bias per Bereiter et al 2015. It replaces the older version of the data from Luthi et al 2008.

I have converted this excel file to a csv format so that it will be easier to use and all so all of the data will be in one uniform format.


In [450]:
# Read and store content of an excel file  
IPCC1 = pd.read_excel ("IPCCreport1.xlsx", sheet_name='CO2 Composite') 
  
# Write the dataframe object into csv file 
IPCC1.to_csv ("IPCC.csv") 
    
# read csv file and convert into a dataframe object 
IPCC = pd.read_csv("IPCC.csv", skiprows=15, header=None) # skipping first 15 rows as they do not contain actual data

IPCC = IPCC.iloc[:, 1:] # skipping first column as it just contains the row number; not actual data. The 1 specifies to skip column 0

IPCC.columns = ['Gasage (yr BP)', 'CO2 (ppmv)', 'sigma mean CO2 (ppmv)'] # setting column titles

IPCC.to_csv("IPCC.csv", index=None) # saving the csv file back to the dataframe with the adjustments made

#IPCC['Gasage(yr BP)'] = 1950 - IPCC['Gasage (yr BP)']


# Mauna Loa CO2 data

In [451]:
MLC = pd.read_csv('MaunaLoaCO2.txt', skiprows=45, header=None, delim_whitespace=True, engine='python') 
#true showing that the data in the text file is separated by whitespace
MLC = MLC.iloc[:, :-1 ]
MLC.columns = ['Year', 'Mean CO2']

MLC.to_csv (r'MaunaLoaCO2.csv', index=None)



# Temperature data

In [452]:
columns = ['Bag', 'ztop','Age','Deuterium', 'Temperature']

data = pd.read_csv('EpicaTemp.txt', skiprows=92, delim_whitespace=True, names=columns, engine='python') 
#true showing that the data in the text file is separated by whitespace

display(data)

df = data[['Age', 'Temperature']] # only want columns with Age and Temperature (other columns contain not applicable)

df.to_csv (r'EpicaTemp.csv', index=None)



,Bag,ztop,Age,Deuterium,Temperature
0,1,0.00,-50.00000,NaN,NaN
1,2,0.55,-43.54769,NaN,NaN
2,3,1.10,-37.41829,NaN,NaN
3,4,1.65,-31.61153,NaN,NaN
4,5,2.20,-24.51395,NaN,NaN
...,...,...,...,...,...
5795,5796,3187.25,797408.00000,-440.20,-8.73
5796,5797,3187.80,798443.00000,-439.00,-8.54
5797,5798,3188.35,799501.00000,-441.10,-8.88
5798,5799,3188.90,800589.00000,-441.42,-8.92


In [453]:
Jouzel = pd.read_excel ("JouzelTempData.xlsx", usecols=[2,5]) 

# Write the dataframe object into csv file 
Jouzel.to_csv ("JouzelTempData.csv") 
 
# read csv file and convert into a dataframe object 
Jouzel = pd.read_csv("JouzelTempData.csv") 

Jouzel.drop('Unnamed: 0', axis=1, inplace=True) # removing unwanted index
#Jouzel = Jouzel.sort_values('EDC3béta', ascending=False)

Jouzel.to_csv("JouzelTempData.csv", index=False) # saving the csv file back to the dataframe with the adjustments made

Jouzel['Year'] = 1950 - Jouzel['EDC3béta'] # to get year in actual values as it was measured in years since 1950

Jouzel.to_csv("JouzelTempData_Year.csv", index=False)

In [454]:

 
# Below are the years and the values taken from Dome C II - 
# Converted from celcius to Degrees Kelvin from intermittent years from 


MaunaLoaTemp = 
     

with open('JouzelTempData.csv', 'a', newline="") as file:
   writer = csv.writer(file)    
   writer.writerows(values)

SyntaxError: invalid syntax (2327521878.py, line 7)

merge mauna loa for temperature and co2 to get most recent year